# Pre-requisite

The following notebook assumes you have [configured remotes for rclone](../aws/rclone/).

We will assume the following variables. It will be loaded via `source .env` or other means:


In [1]:
# .env
# For actual .env, remove prefixing `%env `

# rclone remotes
# configured with STANDARD or INTELLIGENT_TIERING storage cls
%env REMOTE_AWS=aws
# configured with DEEP_GLACIER as storage class
%env REMOTE_AWS_DEEP=aws_deep
%env REMOTE_GDRIVE=gdrive

# root folders & test objects
# Best practice: should only be bucket name, no nested folder
%env ROOT_AWS=test-bucket
%env ROOT_LOCAL=test_objects
%env ROOT_GDRIVE=ronaldstouffer

%env TEST_FOLDER=Becki_Port_april2021

# configure some flags for rclone
# can also put --dry-run or --interactive to lessen the effect on actual drives/buckets
# can put -vv for more diagnostics
%env _rclone=./rclone

# before any codeblock:
# !source .env
!_rclone=${_rclone:-"rclone"}

env: REMOTE_AWS=aws
env: REMOTE_AWS_DEEP=aws_deep
env: REMOTE_GDRIVE=gdrive
env: ROOT_AWS=test-bucket
env: ROOT_LOCAL=test_objects
env: ROOT_GDRIVE=ronaldstouffer
env: TEST_FOLDER=Becki_Port_april2021
env: _rclone=./rclone


# Safe file manipulation with `--dry-run` and `--interactive` flags

These flags are useful to test runs before doing the actual storage management operation.

We don't want to play around with [`rclone purge`](#deleting-files) by itself then mistakenly deleting everything.

`--interactive`: prompts you before doing "destructive" operation

In [2]:
%%bash
rclone --interactive copy . ../ < <(echo "q")
rclone --interactive delete . < <(echo "q")
rclone --interactive purge . < <(echo "q")

rclone: copy "rclone"?
y) Yes, this is OK (default)
n) No, skip this
s) Skip all copy operations with no more questions
!) Do all copy operations with no more questions
q) Exit rclone now.


2022/06/14 18:11:55 NOTICE: Quitting rclone now


y/n/s/!/q> rclone: delete "README.md"?
y) Yes, this is OK (default)
n) No, skip this
s) Skip all delete operations with no more questions
!) Do all delete operations with no more questions
q) Exit rclone now.


2022/06/14 18:11:56 NOTICE: Quitting rclone now


y/n/s/!/q> rclone: purge directory "Local file system at /mnt/c/Users/_nhitrl/repos/git_server/hpc_works/tier-2-storage-management/rclone_commands"?
y) Yes, this is OK (default)
n) No, skip this
s) Skip all purge directory operations with no more questions
!) Do all purge directory operations with no more questions
q) Exit rclone now.


2022/06/14 18:11:56 NOTICE: Quitting rclone now


y/n/s/!/q> 

`--dry-run`: Gives you an example output of the operation. Doesn't do any "destructive" command.

In [ ]:
%%bash
ls -ltr .
rclone --dry-run . this/path/doesnt/get/made
rclone --dry-run delete .
rclone --dry-run purge .

ls -ltr .

# Listing files

`rclone lsl [remote:]path`: recursive, shows modification time, size, path

In [3]:
!$_rclone lsl .

      123 2022-06-14 15:44:27.462818900 README.md
      183 2022-06-14 16:41:58.720359000 rclone
     9184 2022-06-14 18:11:50.149206200 standard_ops.ipynb


# Copying from one place to another

`rclone copy [remote_src:]src_dir [remote_dest:]dest_dir`
- If no remote is specified, then it means we're performing on local filesystem.
- `rclone copy` works as if you put a trailing `/`
- rclone avoids transfering files that are identical from src and dest
  - Size & (modification time or MD5 SUM)

In [4]:
%%bash
# trailing / demonstration
mkdir abc
touch abc/d.txt abc/hello-world.txt
# abc/d.txt  abc/hello-world.txt
$_rclone copy abc def

ls -ltr abc def
rm -rf def

abc:
total 0
-rwxrwxrwx 1 ubuntu_admin ubuntu_admin 0 Jun 14 18:11 d.txt
-rwxrwxrwx 1 ubuntu_admin ubuntu_admin 0 Jun 14 18:11 hello-world.txt

def:
total 0
-rwxrwxrwx 1 ubuntu_admin ubuntu_admin 0 Jun 14 18:11 hello-world.txt
-rwxrwxrwx 1 ubuntu_admin ubuntu_admin 0 Jun 14 18:11 d.txt


The copy results will be: `def/d.txt  def/hello-world.txt`, equivalent to:

```bash
rclone copy abc/ def/
mkdir def; cp -r abc/ def/
mkdir def; cp abc/* def/
```

In [5]:
%%bash
$_rclone copy abc/ def/
expected=$(ls -ltr abc def)
rm -rf def

mkdir def; cp -r abc/ def/
diff=$(sdiff <(ls -ltr abc def) <(echo $expected))
if [ $? ]; then 
    echo ok; 
else 
    echo $diff; 
fi
rm -rf def

mkdir def; cp abc/* def/
diff=$(sdiff <(ls -ltr abc def) <(echo $expected))
if [ $? ]; then 
    echo ok; 
else 
    echo $diff; 
fi
rm -rf def abc

ok
ok


# Deleting files

**NOTE: Be very careful when deleting files, especially with the purge command. If possible, do try this with the `--interactive` flag or test run with `--dry-run` first!

2 commands:
- `rclone delete [remote:]path/to/dir`: deletes the files in given path. Doesn't delete directory structure
- `rclone purge [remote:]path/to/dir`: acts like `rm -rf path/to/dir`.
  - Cannot use selections (because it's deleting the whole directory)

